In [9]:
import requests
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

In [10]:
def get_team_links(league_year):
    league_url = f'https://fbref.com/en/comps/9/{league_year}/{league_year}-Premier-League-Stats' 
    data = requests.get(league_url)
    print(data)
    soup = BeautifulSoup(data.text)
    league_standings = soup.select('table.stats_table')[0]
    links = league_standings.find_all('a')
    links = [l.get('href') for l in links]
    links = [ l for l in links if '/squads/' in l]
    team_links = [f'https://fbref.com{l}' for l in links ]
    return team_links 

In [11]:
years = ['2023-2024']

In [12]:
team_years  = []
for yr in years:
    team_years.append(get_team_links(yr))

<Response [429]>


IndexError: list index out of range

In [146]:
team_years

[['https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats',
  'https://fbref.com/en/squads/361ca564/Tottenham-Hotspur-Stats',
  'https://fbref.com/en/squads/822bd0ba/Liverpool-Stats',
  'https://fbref.com/en/squads/7c21e445/West-Ham-United-Stats',
  'https://fbref.com/en/squads/18bb7c10/Arsenal-Stats',
  'https://fbref.com/en/squads/d07537b9/Brighton-and-Hove-Albion-Stats',
  'https://fbref.com/en/squads/47c64c55/Crystal-Palace-Stats',
  'https://fbref.com/en/squads/cd051869/Brentford-Stats',
  'https://fbref.com/en/squads/e4a775cb/Nottingham-Forest-Stats',
  'https://fbref.com/en/squads/8602292d/Aston-Villa-Stats',
  'https://fbref.com/en/squads/19538871/Manchester-United-Stats',
  'https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats',
  'https://fbref.com/en/squads/fd962109/Fulham-Stats',
  'https://fbref.com/en/squads/b2b47a98/Newcastle-United-Stats',
  'https://fbref.com/en/squads/8cec06e1/Wolverhampton-Wanderers-Stats',
  'https://fbref.com/en/squads/4ba7cbea/Bournemouth-Stats'

In [147]:
def get_players_links(team_urls):
    all_player_links = []
    for i in range(len(team_urls)):
        data = requests.get(team_urls[i])
        soup = BeautifulSoup(data.text)
        matchlog = soup.select('table.stats_table')[0]
        player_links = matchlog.find_all('a')    
        player_links = [l.get('href') for l in player_links]
        player_links = [l for l in player_links if '/matchlogs/' in l]
        player_links = [f'https://fbref.com{l}' for l in player_links]
        all_player_links.extend(player_links)
    return all_player_links

In [148]:
def get_season_stats_per_player(all_player_links):
    dataframes = [] 
    from time import sleep
    import pandas as pd
    for i in range (len(all_player_links)):
        name, season = get_name_and_season(all_player_links[i])
        print(f"{name} {season} Premier league season.. {(i+1) / len(all_player_links)} Done" )
        player_stats_link = all_player_links[i]
        player_data  = requests.get(player_stats_link)
        #print(player_data)
        player_matches = pd.read_html(player_data.text, match="Match Logs")[0]
        player_matches.columns = player_matches.columns.droplevel()
        player_df = player_matches[player_matches['Comp'] == 'Premier League']
        player_df['name'] = name 
        player_df['season'] = season
        dataframes.append(player_df)
        sleep(5)
    players_df = pd.concat(dataframes)
    return players_df 

In [149]:
def get_name_and_season(url):
    name = url.split('/')[-1].replace("-Match-Logs","")
    name = name.replace('-', ' ')
    season = url.split('/')[7]
    season = season.replace('-', '/')
    #print(f"{name} {season} premier league")
    return name, season 

In [150]:
def get_players_overall_stats(team_urls):
    all_player_links = []
    for i in range(len(team_urls)):
        data = requests.get(team_urls[i])
        soup = BeautifulSoup(data.text)
        squadlog = soup.select('table.stats_table')[0]
        squad_info = pd.read_html(squadlog.text, match="Standard Stats")[0]
        all_player_links.extend(squad_info)
    return all_player_links

In [151]:
all_season_stats = [] 
for i in range (len(team_years)):
    players_link = get_players_links(team_years[i])
    all_season_stats.append( get_season_stats_per_player(players_link)) 


Ederson 2023/2024 Premier league season.. 0.0025380710659898475 Done
Julian Alvarez 2023/2024 Premier league season.. 0.005076142131979695 Done
Kyle Walker 2023/2024 Premier league season.. 0.007614213197969543 Done
Erling Haaland 2023/2024 Premier league season.. 0.01015228426395939 Done
Rodri 2023/2024 Premier league season.. 0.012690355329949238 Done
Mateo Kovacic 2023/2024 Premier league season.. 0.015228426395939087 Done
Phil Foden 2023/2024 Premier league season.. 0.017766497461928935 Done
Manuel Akanji 2023/2024 Premier league season.. 0.02030456852791878 Done
Ruben Dias 2023/2024 Premier league season.. 0.02284263959390863 Done
Nathan Ake 2023/2024 Premier league season.. 0.025380710659898477 Done
Bernardo Silva 2023/2024 Premier league season.. 0.027918781725888325 Done
Josko Gvardiol 2023/2024 Premier league season.. 0.030456852791878174 Done
Jack Grealish 2023/2024 Premier league season.. 0.03299492385786802 Done
Rico Lewis 2023/2024 Premier league season.. 0.035532994923857

Joel Veltman 2023/2024 Premier league season.. 0.2893401015228426 Done
Mahmoud Dahoud 2023/2024 Premier league season.. 0.2918781725888325 Done
Julio Cesar Enciso 2023/2024 Premier league season.. 0.29441624365482233 Done
Adam Lallana 2023/2024 Premier league season.. 0.2969543147208122 Done
Simon Adingra 2023/2024 Premier league season.. 0.29949238578680204 Done
Tariq Lamptey 2023/2024 Premier league season.. 0.3020304568527919 Done
Joachim Andersen 2023/2024 Premier league season.. 0.30456852791878175 Done
Jordan Ayew 2023/2024 Premier league season.. 0.30710659898477155 Done
Cheick Doucoure 2023/2024 Premier league season.. 0.3096446700507614 Done
Eberechi Eze 2023/2024 Premier league season.. 0.31218274111675126 Done
Sam Johnstone 2023/2024 Premier league season.. 0.3147208121827411 Done
Jefferson Lerma 2023/2024 Premier league season.. 0.31725888324873097 Done
Tyrick Mitchell 2023/2024 Premier league season.. 0.3197969543147208 Done
Marc Guehi 2023/2024 Premier league season.. 0.3

Moises Caicedo 2023/2024 Premier league season.. 0.5786802030456852 Done
Carney Chukwuemeka 2023/2024 Premier league season.. 0.5812182741116751 Done
Reece James 2023/2024 Premier league season.. 0.583756345177665 Done
Mykhailo Mudryk 2023/2024 Premier league season.. 0.5862944162436549 Done
Noni Madueke 2023/2024 Premier league season.. 0.5888324873096447 Done
Ian Maatsen 2023/2024 Premier league season.. 0.5913705583756346 Done
Cole Palmer 2023/2024 Premier league season.. 0.5939086294416244 Done
Mason Burstow 2023/2024 Premier league season.. 0.5964467005076142 Done
Lesley Ugochukwu 2023/2024 Premier league season.. 0.5989847715736041 Done
Issa Diop 2023/2024 Premier league season.. 0.6015228426395939 Done
Bernd Leno 2023/2024 Premier league season.. 0.6040609137055838 Done
Antonee Robinson 2023/2024 Premier league season.. 0.6065989847715736 Done
Kenny Tete 2023/2024 Premier league season.. 0.6091370558375635 Done
Harry Wilson 2023/2024 Premier league season.. 0.6116751269035533 Do

Arnaut Danjuma 2023/2024 Premier league season.. 0.8705583756345178 Done
Michael Keane 2023/2024 Premier league season.. 0.8730964467005076 Done
Jarrad Branthwaite 2023/2024 Premier league season.. 0.8756345177664975 Done
Alex Iwobi 2023/2024 Premier league season.. 0.8781725888324873 Done
Lewis Dobbin 2023/2024 Premier league season.. 0.8807106598984772 Done
Neal Maupay 2023/2024 Premier league season.. 0.883248730964467 Done
Beto 2023/2024 Premier league season.. 0.8857868020304569 Done
Dominic Calvert Lewin 2023/2024 Premier league season.. 0.8883248730964467 Done
Youssef Chermiti 2023/2024 Premier league season.. 0.8908629441624365 Done
Dwight McNeil 2023/2024 Premier league season.. 0.8934010152284264 Done
Thomas Cannon 2023/2024 Premier league season.. 0.8959390862944162 Done
Ben Godfrey 2023/2024 Premier league season.. 0.8984771573604061 Done
Tyler Onyango 2023/2024 Premier league season.. 0.9010152284263959 Done
Amarii Bell 2023/2024 Premier league season.. 0.9035532994923858 

In [196]:
len(all_season_stats)

1

In [2]:
import pandas as pd
player_data = pd.concat(all_season_stats)

NameError: name 'all_season_stats' is not defined

In [3]:
player_data

NameError: name 'player_data' is not defined

In [250]:
player_data = player_data[player_data['Pos'] != 'On matchday squad, but did not play']

In [251]:
s = player_data 


In [256]:
player_data.columns
player_data = player_data[['name', 'season', 'Date', 'Day', 'Comp', 'Round', 'Venue', 'Result', 'Squad', 'Opponent',
       'Start', 'Pos', 'Min', 'Gls', 'Ast', 'PK', 'PKatt', 'Sh', 'SoT', 'CrdY',
       'CrdR', 'Touches', 'Tkl', 'Int', 'Blocks', 'xG', 'npxG', 'xAG', 'SCA',
       'GCA', 'Cmp', 'Att', 'Cmp%', 'PrgP', 'Carries', 'PrgC', 'Att', 'Succ',
       'Match Report']]


In [258]:
player_data['Pos']

1            [GK]
3            [GK]
4            [GK]
5            [GK]
1    [LM, AM, FW]
         ...     
4            [CM]
0            [FW]
0            [FW]
4            [CM]
0            [FW]
Name: Pos, Length: 1180, dtype: object

In [260]:
#player_data['Pos'] = player_data['Pos'].str.split(',')
filtered_players = player_data[player_data['Pos'].apply(lambda pos_list: 'FW' in pos_list)]
#player_with_appearances.columns
# Duplicate rows for each position


1    [LM, AM, FW]
1            [FW]
3            [FW]
4            [FW]
5            [FW]
         ...     
4            [FW]
0            [FW]
0            [FW]
0            [FW]
0            [FW]
Name: Pos, Length: 177, dtype: object

In [263]:
filtered_players.to_csv("PL2022-23Explodeseason.csv")


In [265]:
second_df = pd.read_csv('players_2223.csv')


In [266]:
second_df['name'] = second_df['first_name'] + " " + second_df['second_name']

In [268]:
second_df = second_df.drop(['first_name', 'second_name'], axis = 'columns')

In [269]:
import pandas as pd

# Sample DataFrame 1 (with player names)

df1 = filtered_players


# Filter the second DataFrame to include only forwards
filtered_df2 = second_df[second_df['element_type'] == 'FWD']

# Merge the two DataFrames on the 'Player' column
result_df = df1.merge(filtered_df2, on='name', how='inner')

print(result_df)


              name     season        Date  Day            Comp        Round  \
0   Erling Haaland  2023/2024  2023-08-11  Fri  Premier League  Matchweek 1   
1   Erling Haaland  2023/2024  2023-08-19  Sat  Premier League  Matchweek 2   
2   Erling Haaland  2023/2024  2023-08-27  Sun  Premier League  Matchweek 3   
3   Erling Haaland  2023/2024  2023-09-02  Sat  Premier League  Matchweek 4   
4       Cody Gakpo  2023/2024  2023-08-27  Sun  Premier League  Matchweek 3   
..             ...        ...         ...  ...             ...          ...   
90     Lyle Foster  2023/2024  2023-09-02  Sat  Premier League  Matchweek 4   
91    Zeki Amdouni  2023/2024  2023-08-11  Fri  Premier League  Matchweek 1   
92    Zeki Amdouni  2023/2024  2023-08-27  Sun  Premier League  Matchweek 3   
93    Zeki Amdouni  2023/2024  2023-09-02  Sat  Premier League  Matchweek 4   
94   Jay Rodriguez  2023/2024  2023-08-27  Sun  Premier League  Matchweek 3   

   Venue Result            Squad         Opponent  

In [270]:
result_df.to_csv("FixedForwards.csv")

In [140]:
player_data.to_csv("PLALLEverySeason-withappearances.csv")

In [59]:
player_data.to_excel("PLALLEverySeason-withappearances.xlsx")

In [191]:
data  = pd.read_csv("PL2022-23season.csv")
data

,Unnamed: 0,Date,Day,Comp,Round,Venue,Result,Squad,Opponent,Start,...,Att,Cmp%,PrgP,Carries,PrgC,Att.1,Succ,Match Report,name,season
0,1,2023-08-11,Fri,Premier League,Matchweek 1,Away,W 3–0,Manchester City,Burnley,Y,...,11.0,81.8,1.0,15.0,1.0,0.0,0.0,Match Report,Erling Haaland,2023/2024
1,3,2023-08-19,Sat,Premier League,Matchweek 2,Home,W 1–0,Manchester City,Newcastle Utd,Y,...,11.0,54.5,1.0,15.0,3.0,1.0,0.0,Match Report,Erling Haaland,2023/2024
2,4,2023-08-27,Sun,Premier League,Matchweek 3,Away,W 2–1,Manchester City,Sheffield Utd,Y,...,11.0,54.5,0.0,15.0,0.0,1.0,1.0,Match Report,Erling Haaland,2023/2024
3,5,2023-09-02,Sat,Premier League,Matchweek 4,Home,W 5–1,Manchester City,Fulham,Y,...,6.0,66.7,1.0,9.0,0.0,1.0,0.0,Match Report,Erling Haaland,2023/2024
4,4,2023-09-02,Sat,Premier League,Matchweek 4,Away,W 5–2,Tottenham,Burnley,Y*,...,18.0,83.3,5.0,18.0,3.0,1.0,0.0,Match Report,Son Heung min,2023/2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,4,2023-09-02,Sat,Premier League,Matchweek 4,Home,L 2–5,Burnley,Tottenham,Y,...,16.0,87.5,3.0,17.0,2.0,2.0,1.0,Match Report,Zeki Amdouni,2023/2024
140,0,2023-08-11,Fri,Premier League,Matchweek 1,Home,L 0–3,Burnley,Manchester City,N,...,5.0,80.0,0.0,7.0,1.0,4.0,0.0,Match Report,Benson Manuel,2023/2024
141,0,2023-08-11,Fri,Premier League,Matchweek 1,Home,L 0–3,Burnley,Manchester City,N,...,12.0,83.3,0.0,6.0,0.0,0.0,0.0,Match Report,Anass Zaroury,2023/2024
142,0,2023-08-27,Sun,Premier League,Matchweek 3,Home,L 1–3,Burnley,Aston Villa,N,...,5.0,80.0,1.0,5.0,0.0,0.0,0.0,Match Report,Aaron Ramsey,2023/2024


In [192]:
player_data.columns
player_data = data [['Round', 'Min','Pos', 'Sh', 'SoT', 'Touches', 'Tkl', 'Int', 'Blocks', 'xG',
       'npxG', 'xAG', 'SCA', 'GCA', 'Cmp', 'Att', 'Cmp%', 'PrgP', 'Carries',
       'PrgC', 'Att', 'Succ', 'name']]


,Round,Min,Pos,Sh,SoT,Touches,Tkl,Int,Blocks,xG,...,GCA,Cmp,Att,Cmp%,PrgP,Carries,PrgC,Att,Succ,name
0,Matchweek 1,79.0,FW,4.0,2.0,18.0,0.0,0.0,0.0,0.6,...,0.0,9.0,11.0,81.8,1.0,15.0,1.0,11.0,0.0,Erling Haaland
1,Matchweek 2,90.0,FW,4.0,2.0,22.0,0.0,1.0,1.0,0.4,...,0.0,6.0,11.0,54.5,1.0,15.0,3.0,11.0,0.0,Erling Haaland
2,Matchweek 3,90.0,FW,7.0,3.0,23.0,0.0,0.0,0.0,2.1,...,1.0,6.0,11.0,54.5,0.0,15.0,0.0,11.0,1.0,Erling Haaland
3,Matchweek 4,90.0,FW,3.0,2.0,17.0,0.0,0.0,0.0,1.6,...,1.0,4.0,6.0,66.7,1.0,9.0,0.0,6.0,0.0,Erling Haaland
4,Matchweek 4,71.0,FW,5.0,3.0,27.0,0.0,0.0,0.0,1.1,...,1.0,15.0,18.0,83.3,5.0,18.0,3.0,18.0,0.0,Son Heung min
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,Matchweek 4,59.0,FW,1.0,0.0,24.0,4.0,0.0,1.0,0.0,...,0.0,14.0,16.0,87.5,3.0,17.0,2.0,16.0,1.0,Zeki Amdouni
140,Matchweek 1,17.0,FW,0.0,0.0,9.0,0.0,0.0,0.0,0.0,...,0.0,4.0,5.0,80.0,0.0,7.0,1.0,5.0,0.0,Benson Manuel
141,Matchweek 1,29.0,FW,1.0,0.0,13.0,0.0,0.0,0.0,0.0,...,0.0,10.0,12.0,83.3,0.0,6.0,0.0,12.0,0.0,Anass Zaroury
142,Matchweek 3,23.0,FW,0.0,0.0,6.0,0.0,0.0,0.0,0.0,...,0.0,4.0,5.0,80.0,1.0,5.0,0.0,5.0,0.0,Aaron Ramsey


In [193]:

player_data = player_data.drop(['Pos'], axis="columns")
grouped = player_data.groupby('name')
average_stats = grouped.mean()

ValueError: columns must be unique

In [185]:
average_stats

,Min,Sh,SoT,Touches,Tkl,Int,Blocks,xG,npxG,xAG,SCA,GCA,Cmp,Att,Cmp%,PrgP,Carries,PrgC,Att,Succ
name,,,,,,,,,,,,,,,,,,,,
Aaron Ramsey,23.000000,0.000000,0.000000,6.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,4.000000,5.000000,80.000000,1.0,5.000000,0.000000,5.000000,0.000000
Abdoulaye Doucoure,90.000000,3.000000,2.000000,46.500000,0.5,0.00,2.000000,0.850000,0.850000,0.500000,3.000000,0.50,29.500000,37.000000,80.100000,3.0,25.500000,3.000000,37.000000,0.500000
Adam Lallana,10.000000,0.000000,0.000000,16.000000,1.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,14.000000,14.000000,100.000000,1.0,7.000000,0.000000,14.000000,0.000000
Aleksandar Mitrovic,33.000000,2.000000,1.000000,12.000000,0.0,0.00,0.000000,0.200000,0.200000,0.000000,2.000000,1.00,4.000000,8.000000,50.000000,1.0,5.000000,0.000000,8.000000,3.000000
Alexander Isak,67.666667,1.333333,1.333333,14.666667,0.0,0.00,0.000000,0.333333,0.333333,0.033333,1.666667,0.00,8.000000,10.333333,78.333333,1.0,10.333333,1.000000,10.333333,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Thomas Cannon,1.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,NaN,0.0,0.000000,0.000000,0.000000,0.000000
William Osula,77.666667,1.000000,0.000000,20.000000,0.0,0.00,0.666667,0.066667,0.066667,0.000000,0.333333,0.00,4.666667,9.666667,49.866667,0.0,10.000000,0.666667,9.666667,0.000000
Yoane Wissa,82.250000,3.750000,1.250000,26.750000,0.5,0.25,1.500000,0.600000,0.600000,0.025000,1.250000,0.25,10.500000,14.250000,75.550000,1.0,13.000000,1.500000,14.250000,0.500000


In [186]:
average_stats.reset_index(inplace=True)
average_stats


,name,Min,Sh,SoT,Touches,Tkl,Int,Blocks,xG,npxG,...,SCA,GCA,Cmp,Att,Cmp%,PrgP,Carries,PrgC,Att,Succ
0,Aaron Ramsey,23.000000,0.000000,0.000000,6.000000,0.0,0.00,0.000000,0.000000,0.000000,...,0.000000,0.00,4.000000,5.000000,80.000000,1.0,5.000000,0.000000,5.000000,0.000000
1,Abdoulaye Doucoure,90.000000,3.000000,2.000000,46.500000,0.5,0.00,2.000000,0.850000,0.850000,...,3.000000,0.50,29.500000,37.000000,80.100000,3.0,25.500000,3.000000,37.000000,0.500000
2,Adam Lallana,10.000000,0.000000,0.000000,16.000000,1.0,0.00,0.000000,0.000000,0.000000,...,0.000000,0.00,14.000000,14.000000,100.000000,1.0,7.000000,0.000000,14.000000,0.000000
3,Aleksandar Mitrovic,33.000000,2.000000,1.000000,12.000000,0.0,0.00,0.000000,0.200000,0.200000,...,2.000000,1.00,4.000000,8.000000,50.000000,1.0,5.000000,0.000000,8.000000,3.000000
4,Alexander Isak,67.666667,1.333333,1.333333,14.666667,0.0,0.00,0.000000,0.333333,0.333333,...,1.666667,0.00,8.000000,10.333333,78.333333,1.0,10.333333,1.000000,10.333333,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,Thomas Cannon,1.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,NaN,0.0,0.000000,0.000000,0.000000,0.000000
61,William Osula,77.666667,1.000000,0.000000,20.000000,0.0,0.00,0.666667,0.066667,0.066667,...,0.333333,0.00,4.666667,9.666667,49.866667,0.0,10.000000,0.666667,9.666667,0.000000
62,Yoane Wissa,82.250000,3.750000,1.250000,26.750000,0.5,0.25,1.500000,0.600000,0.600000,...,1.250000,0.25,10.500000,14.250000,75.550000,1.0,13.000000,1.500000,14.250000,0.500000
63,Youssef Chermiti,25.000000,1.000000,0.000000,10.000000,0.0,0.00,0.000000,0.100000,0.100000,...,0.000000,0.00,4.000000,5.000000,80.000000,1.0,8.000000,0.000000,5.000000,0.000000


In [188]:
mean_Comp = average_stats['Cmp%'].mean()
average_stats['Cmp%'].fillna(mean_Comp, inplace=True)

In [189]:
average_stats.to_csv("FinalCleanedValues-2223SeasonPl.csv")

In [190]:
average_stats.columns

Index(['name', 'Min', 'Sh', 'SoT', 'Touches', 'Tkl', 'Int', 'Blocks', 'xG',
       'npxG', 'xAG', 'SCA', 'GCA', 'Cmp', 'Att', 'Cmp%', 'PrgP', 'Carries',
       'PrgC', 'Att', 'Succ'],
      dtype='object')